<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

# Accelerating End-to-End Data Science Workflows # 

## 05 - Grouping ##

**Table of Contents**
<br>
This notebook discusses and demonstrates how grouping in used in data science. This notebook covers the below sections: 
1. [Grouping](#Grouping)
    * [Split, Apply, and Combine](#Split,-Apply,-and-Combine)
    * [Exercise #1 - Average Age Per County](#Exercise-#1---Average-Age-Per-County)
2. [Binning](#Binning)
    * [Exercise #2 - Using the Profiler](#Exercise-#2---Using-the-Profiler)
3. [Advanced Groupby Operations](#Advanced-Groupby-Operations)
    * [`.apply()`](#.apply())
    * [`.transform()`](#.transform())
4. [Pivot Table](#Pivot-Table)

## Grouping ##
In data science, we often would like to split data into groups and perform further analysis on them such as: 
* Aggregate based on the grouping
* Compare metrics across different groups
* Understand patterns in data across different groups
* Remove duplicates or fill missing values based on group-level information
* Create new features based on group-level statistics
* Integrate with visualization

Below we load in our dataset. 

In [1]:
# DO NOT CHANGE THIS CELL
%load_ext cudf.pandas
import pandas as pd
import time

In [2]:
# DO NOT CHANGE THIS CELL
dtype_dict={
    'age': 'int8', 
    'sex': 'category', 
    'county': 'category', 
    'lat': 'float32', 
    'long': 'float32', 
    'name': 'category'
}
        
df=pd.read_csv('./data/uk_pop.csv', dtype=dtype_dict)
df.head()

,age,sex,county,lat,long,name
0,0,m,DARLINGTON,54.533638,-1.524400,FRANCIS
1,0,m,DARLINGTON,54.426254,-1.465314,EDWARD
2,0,m,DARLINGTON,54.555199,-1.496417,TEDDY
3,0,m,DARLINGTON,54.547909,-1.572342,ANGUS
4,0,m,DARLINGTON,54.477638,-1.605995,CHARLIE


## Split, Apply, and Combine ##
We use the `.groupby()` method to to group large amounts of data and compute operations on these groups. A groupby operation involves some combination of splitting the object, applying a function, and combining the results. cuDF implements record grouping in a manner comparable to Pandas, but with some notable differences. 

<p><img src='images/groupby.png' width=720></p>

cuDF supports a number of common `DataFrameGroupBy` computations and descriptive statistics, such as `.size()`, `.mean()`, `.count()`, `.cov()`, `.cumprod()`, `.cumsum()`, `.max()`, `.min()`, `.nunique()`. 

**Note**: More information about how `.groupby()` behaves for pandas and how it differs from cuDF can be found in the links below: 
* [pandas](https://pandas.pydata.org/docs/user_guide/groupby.html)
* [cuDF](https://docs.rapids.ai/api/cudf/stable/user_guide/groupby/)

Below we find the number of people in each county. 

In [3]:
# DO NOT CHANGE THIS CELL
df.groupby('county').size()

county
BARKING AND DAGENHAM            211998
BARNET                          392140
BARNSLEY                        245199
BATH AND NORTH EAST SOMERSET    192106
BEDFORD                         171623
                                 ...  
WOKINGHAM                       167979
WOLVERHAMPTON                   262008
WORCESTERSHIRE                  592057
WREXHAM                         136126
YORK                            209893
Length: 171, dtype: int64

**Note**: The results is unsorted. We can sort the output using the `.sort_index()` or `.sort_values()` method. 

Below we count the number of people with the most and least popular names. 

In [4]:
# DO NOT CHANGE THIS CELL
df.groupby('name').size().sort_values()

name
AKASHDEEP       213
DALHA           214
BOGOMIL         215
REMMY           217
KAIYAAN         219
              ...  
GEORGE       459096
HARRY        459346
AMELIA       460659
OLIVIA       483789
OLIVER       576135
Length: 13212, dtype: int64

Below we find the approximate centers of each county using `.groupby().mean()`. When performing groupby operations, we should **only** include columns that are being used. 

In [5]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

county_center_df=df[['county', 'lat', 'long']].groupby('county')[['lat', 'long']].mean()
display(county_center_df)

,lat,long
county,,
BARKING AND DAGENHAM,51.621072,0.129583
BARNET,51.812547,-0.218213
BARNSLEY,53.571882,-1.548719
BATH AND NORTH EAST SOMERSET,51.354919,-2.486676
BEDFORD,52.145470,-0.454974
...,...,...
WOKINGHAM,51.459653,-0.899370
WOLVERHAMPTON,52.716856,-2.127594
WORCESTERSHIRE,52.057971,-2.209186


                                                                                                                   
                                             Total time elapsed: 0.577 seconds                                     
                                                                                                                   
                                                           Stats                                                   
                                                                                                                   
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Line no. ┃ Line                                                                     ┃ GPU TIME(s) ┃ CPU TIME(s) ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 3        │     county_center_df=df[['county', 'lat', 'long']].groupby('county')[['… │ 0.097302883 │             │
│          │                                                                          │             │             │
│ 4        │     display(county_center_df)                                            │ 0.179342007 │             │
│          │                                                                          │             │             │
└──────────┴──────────────────────────────────────────────────────────────────────────┴─────────────┴─────────────┘

In [6]:
# DO NOT CHANGE THIS CELL
county_center_df.columns=['lat_county_center', 'long_county_center']
county_center_df.to_csv('county_centroid.csv')

### Exercise #1 - Average Age Per County ###
We would like to find the average age for each county. We will need to use both `.groupby()` and `.sort_values()`. Using the `.mean()` method on the data grouped by `county`, identify the 5 counties with the highest average age. 

**Instructions**: <br>
* Modify the `<FIXME>` only and execute the below cell find the average age for each county. 

In [7]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL
display(
    df[['county', 'age']].groupby('county')['age']\
                         .mean()\
                         .sort_values(ascending=False)\
                         .head()
)

county
DORSET             46.577193
ISLE OF WIGHT      46.149253
CONWY              45.854473
POWYS              45.849366
ISLES OF SCILLY    45.467440
Name: age, dtype: float64

                                                                                                      
                                      Total time elapsed: 0.768 seconds                               
                                                                                                      
                                                    Stats                                             
                                                                                                      
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Line no. ┃ Line                                                        ┃ GPU TIME(s) ┃ CPU TIME(s) ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 2        │     display(                                                │ 0.019252037 │ 0.013523365 │
│          │                                                             │             │             │
│ 3        │         df[['county', 'age']].groupby('county')['age']\     │ 0.012133539 │             │
│          │                                                             │             │             │
│ 4        │                              .mean()\                       │ 0.059768666 │             │
│          │                                                             │             │             │
│ 5        │                              .sort_values(ascending=False)\ │             │             │
│          │                                                             │             │             │
│ 6        │                              .head()                        │ 0.003951281 │             │
│          │                                                             │             │             │
└──────────┴─────────────────────────────────────────────────────────────┴─────────────┴─────────────┘

Click ... for solution. 

## Binning ##
When grouping continuous numerical data, it is sometimes helpful to bin numbers into discrete intervals or buckets. There are primarily two ways of binning: 
* Equal-width binning: divide the range into equal-sized intervals
* Custom binning: define custom bins based on domain knowledge or specific criteria

The `.cut()` function can be used to bin values into discrete intervals

In [8]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
df['age_bucket']=pd.cut(df['age'].values, bins=bins, right=True, include_lowest=True, labels=False)
display(df.groupby('age_bucket').size())

age_bucket
0    7874941
1    6630853
2    7758863
3    7691036
4    7598003
5    7712976
6    6124213
7    4530946
8    2558063
dtype: int64

                                                                                                                   
                                             Total time elapsed: 0.530 seconds                                     
                                                                                                                   
                                                           Stats                                                   
                                                                                                                   
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Line no. ┃ Line                                                                     ┃ GPU TIME(s) ┃ CPU TIME(s) ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 3        │     bins=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]                    │             │             │
│          │                                                                          │             │             │
│ 4        │     df['age_bucket']=pd.cut(df['age'].values, bins=bins, right=True, in… │ 0.145493617 │             │
│          │                                                                          │             │             │
│ 5        │     display(df.groupby('age_bucket').size())                             │ 0.048880802 │ 0.006587887 │
│          │                                                                          │             │             │
└──────────┴──────────────────────────────────────────────────────────────────────────┴─────────────┴─────────────┘

### Exercise #2 - Using the Profiler ###
cuDF pandas will attempt to use the GPU whenever possible and fall back to CPU for certain operations. Running the code with the `cudf.pandas.line_profile` magic command generates a report showing which operations used the GPU and which used the CPU. 

**Instructions**: <br>
* Notice that the below cell is a very similar operation as before, except that it uses the `range()` function for the `bins` parameter. As it stands, this is not supported in cuDF. 
* Execute the cell below to run the binning operation on the CPU.
* Compare the time it takes to run the similar operation above. 

In [9]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

df['age_bucket']=pd.cut(df['age'].values, bins=range(0, 100, 10), right=True, include_lowest=True, labels=False)
display(df.groupby('age_bucket').size())

age_bucket
0    7874941
1    6630853
2    7758863
3    7691036
4    7598003
5    7712976
6    6124213
7    4530946
8    2558063
dtype: int64

                                                                                                                   
                                             Total time elapsed: 1.715 seconds                                     
                                                                                                                   
                                                           Stats                                                   
                                                                                                                   
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Line no. ┃ Line                                                                     ┃ GPU TIME(s) ┃ CPU TIME(s) ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 3        │     df['age_bucket']=pd.cut(df['age'].values, bins=range(0, 100, 10), r… │ 0.332245756 │ 1.014618294 │
│          │                                                                          │             │             │
│ 4        │     display(df.groupby('age_bucket').size())                             │ 0.075676498 │ 0.006589641 │
│          │                                                                          │             │             │
└──────────┴──────────────────────────────────────────────────────────────────────────┴─────────────┴─────────────┘

**Note**: The profiler can help us identify parts of our code that could be rewritten to be more GPU-friendly. 

## Advanced Groupby Operations ##
We can also use function application helpers on `DataFrameGroupBy` instances: 
* `DataFrameGroupby.aggregate()` / `Groupby.agg()`(alias): used when we have specific computation for different columns or more than one computation on the same column
* `DataFrameGroupby.apply()`: used when we want to perform a specific user-defined function to each group
* `DataFrameGroupby.transform()`: used when the resulting values should be broadcast across the whole group and return a same-indexed dataframe

### `.apply()` ###
The `.apply()` method will **sequentially** apply the function group-wise and concatenate the results together. We can pass a callable function to be performed on the entire DataFrame for each group. 

Below we calculate the distance of each person from their respective county center. 

In [10]:
# DO NOT CHANGE THIS CELL

# define distance function
def distance(lat_1, long_1, lat_2, long_2): 
    return ((lat_2-lat_1)**2+(long_2-long_1)**2)**0.5

In [11]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

distance_df=df.groupby('county')[['lat', 'long']].apply(lambda x: distance(x['lat'], x['long'], x['lat'].mean(), x['long'].mean()))
df['R_1']=distance_df.reset_index(level=0, drop=True)

/opt/conda/lib/python3.10/site-packages/cudf/core/groupby/groupby.py:1449: RuntimeWarning: GroupBy.apply() performance scales poorly with number of groups. Got 171 groups. Some functions may perform better by passing engine='jit'
  warnings.warn(


                                                                                                                   
                                            Total time elapsed: 20.249 seconds                                     
                                                                                                                   
                                                           Stats                                                   
                                                                                                                   
┏━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Line no. ┃ Line                                                                     ┃ GPU TIME(s) ┃ CPU TIME(s) ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ 3        │     distance_df=df.groupby('county')[['lat', 'long']].apply(lambda x: d… │ 3.189867688 │             │
│          │                                                                          │             │             │
│ 4        │     df['R_1']=distance_df.reset_index(level=0, drop=True)                │ 0.616267967 │             │
│          │                                                                          │             │             │
└──────────┴──────────────────────────────────────────────────────────────────────────┴─────────────┴─────────────┘

We can also define the function in-line. 

In [ ]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

df['R_2']=df.groupby('county')[['lat', 'long']].apply(lambda x: ((x['lat'].mean()-x['lat'])**2+(x['long'].mean()-x['long'])**2)**0.5).reset_index(level=0, drop=True)

**Note**: This is quite slow due to the iterative nature of the `.apply()` method. 

### `.transform()` ###
The `.transform()` method aggregates each group, and broadcasts the result to the group size, resuliting in a DataFrame that is the same size and index as the input DataFrame. Underneath the hood, the `.transform()` method passes each column individually as a Series to the function. 

Below we group the DataFrame by `county` and transform the columns `lat` and `long` using `mean`. We will subtract the transformed mean from the original columns, then apply the distance formula to calculate the resulting distance.  By keeping the DataFrame the same shape, we can perform cuDF operations quickly, resulting in performance gain. 

In [ ]:
# DO NOT CHANGE THIS CELL
# make data types more precise
df[['lat', 'long']]=df[['lat', 'long']].astype('float64')

In [ ]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

c=['lat', 'long']
df['R_3']=((df[c] - df.groupby('county')[c].transform('mean')) ** 2).sum(axis=1) ** 0.5

In [ ]:
df.head()

Although the `.apply()` method is more flexible and can handle complex operations, it is generally slower. On the other hand, the `.transform()` method can be much faster. When we design the procedures to use vector operations, we will realize significant performance benefits. 

**Note**: `Groupby.apply()` doesn't scale well with the number of groups, therefore this performance difference will be more pronounced with higher number of groups. 

## Pivot Table ##
Pivot tables allow us to summarize and aggregate large datasets into a more manageable format for analysis. When using `DataFrame.pivot_table()`, we provide the `index`, `columns`, and `values` arguments, as well as `aggfunc`. This will group the data based on `index` and `columns`, and perform the aggregation on `values`. We can apply multiple aggregation functions, which is generally faster and more memory-efficient than manual grouping and aggregation for large datasets. 

Below we create a pivot table that counts the number of each sex in each county. Furthermore, we derive the percentage of the total for each county. 

In [ ]:
%%cudf.pandas.line_profile
# DO NOT CHANGE THIS CELL

pvt_tbl=df[['county', 'sex', 'name']].pivot_table(index=['county'], columns=['sex'], values='name', aggfunc='count')
pvt_tbl=pvt_tbl.apply(lambda x: x/sum(x), axis=1)
display(pvt_tbl)

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**Well Done!** Let's move to the [next notebook](1-06_data_visualization.ipynb). 

<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">